In [33]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

/bin/bash: line 1: pip: command not found


  

# Async API


 https://python.langchain.com/docs/modules/agents/how_to/async_agent


In [34]:
import os

#os.environ["OPENAI_API_KEY"] = ""
#os.environ["SERPAPI_API_KEY"] = ""

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [35]:
import asyncio
import time

from langchain.agents import Tool, AgentType, initialize_agent, load_tools

from langchain.agents import initialize_agent
from langchain.llms import AzureOpenAI

In [36]:
#llm = OpenAI(temperature=0)

llm = AzureOpenAI(
    deployment_name="dep1" #gpt-35-turbo
)

/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.6_1/lib/python3.11/site-packages/langchain_community/llms/openai.py:872: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://b0cherch-oai1.openai.azure.com/ to https://b0cherch-oai1.openai.azure.com/openai.
  warnings.warn(
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.6_1/lib/python3.11/site-packages/langchain_community/llms/openai.py:879: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.6_1/lib/python3.11/site-packages/langchain_community/llms/openai.py:887: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specif

In [37]:

questions = [
    "Who won the US Open men's final in 2019? What is his age raised to the 0.334 power?",
    "Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?",
    "Who won the most recent formula 1 grand prix? What is their age raised to the 0.23 power?",
    "Who won the US Open women's final in 2019? What is her age raised to the 0.34 power?",
    "Who is Beyonce's husband? What is his age raised to the 0.19 power?",
]

from langchain.agents import tool

@tool
def get_word_length(string: str) -> int:
    """Returns the length of a word."""
    return len(string)

In [38]:


tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


In [39]:

s = time.perf_counter()
for q in questions:
    agent.run(q)
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")




> Entering new AgentExecutor chain...
 We need to use google_serper to get the name of the tennis player and then use a calculator to find the solution to this math question.
Action: google_serper
Action Input: "us open winner men's 2019"
Observation: Rafael Nadal defeated Daniil Medvedev in the final, 7–5, 6–3, 5–7, 4–6, 6–4 to win the men's singles tennis title at the 2019 US Open. It was his fourth US ... Novak Djokovic and Naomi Osaka were the men's and women's singles defending champions. Neither managed to defend their title, with Djokovic retiring in the ... Relive Rafael Nadal's epic US Open 2019 victory as we ... Duration: 10:04. Posted: Nov 27, 2019. 5 seed Daniil Medvedev, 7-5, 6-3, 5-7, 4-6, 6-4, in the 2019 men's singles final. ... winner punctuated a 28-ball rally on break point for 3-2. Daniil Medvedev vs Rafael Nadal Full Match | 2019 US ... Duration: 4:47:32. Posted: Mar 20, 2020. List of Winners of the USA Open - Men's Singles ; 2020, Dominic Thiem (Austria), Alexan

In [40]:
s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run or loop.run_until_complete
tasks = [agent.arun(q) for q in questions]
await asyncio.gather(*tasks)
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
 I need to find Beyonce's husband first, then I need to do some math
Action: google_serper
Action Input: "beyonce husband" I should use the google_serper tool to answer the first part of the question and the calculator to answer the second part
Action: google_serper
Action Input: US Open women's final 2019 winner This question is asking two things: 1) who won the most recent grand prix, and 2) what is their age raised to the 0.23 power.
Action: google_serper
Action Input: 'formula 1 most recent grand prix winner' I can use google_serper to find out who Olivia Wilde's boyfriend is, and then use calculator to raise his age to the 0.23 power.
Action: google_serper
Action Input: "olivia wilde boyfriend" This requires two steps, first we need to find who won the US Open men's final in 2019, and